<a href="https://colab.research.google.com/github/YunDudali/TAMU-CSCE629-Projet_1/blob/main/Project_1_YunDu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TAMU- CSCE629 Analysis of Algorithms - Spring2023**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 

**Instance and Solution Review**

In [ ]:
import pickle
file = open('/content/drive/MyDrive/629/Project_1/examples_of_instances', 'rb')
info = pickle.load(file)
print(info)  

{'n_list': [94, 96, 77, 98, 105, 130, 89, 123, 95, 98], 'x_list': [[0.6134631912663004, 1.0211871365118395, 2.0729039402348004, 2.2190846695219926, 2.9899022226925003, 3.9995046741727225, 4.680688861335032, 5.357493072598398, 6.216874000016476, 6.5852567085993705, 6.685495200831027, 6.97701878423553, 7.155802032533437, 7.931875348567257, 8.1105392621185, 9.078721848826182, 9.288212011128385, 9.347143655403437, 9.355200454634089, 14.442693707499558, 17.954184837711843, 21.53586433334032, 31.01015218910708, 31.5169356981405, 35.23123929440764, 41.02165347524516, 44.09756245965823, 44.17882021866267, 44.569058765256564, 46.288131393382336, 51.78299095837458, 53.351135002019525, 69.88370821213039, 70.84991999499186, 71.01542066060688, 71.3298145492108, 73.44485723514025, 73.60203799916296, 73.99885931877432, 74.23942386710658, 75.59510051789317, 76.84793251028483, 77.05863585616986, 77.7747653503294, 78.05949289216338, 80.15005308133097, 80.33859556313611, 80.49988013524577, 80.50961450694

In [ ]:
import pickle
file = open('/content/drive/MyDrive/629/Project_1/examples_of_solutions', 'rb')
res = pickle.load(file)
print(res)  

{'k_list': [3, 5, 7, 5, 5, 6, 5, 6, 5, 5], 'last_points_list': [[31, 44, 93], [17, 34, 49, 75, 95], [21, 45, 49, 62, 70, 72, 76], [14, 35, 58, 67, 97], [21, 32, 54, 80, 104], [16, 46, 71, 100, 119, 129], [9, 38, 52, 65, 88], [24, 46, 69, 72, 101, 122], [10, 30, 55, 69, 94], [11, 35, 47, 73, 97]], 'OPT_list': [9955.201272375221, 12127.705100827881, 9763.325183164514, 10943.868071846184, 124.31733328067263, 165.22998494410663, 124.36266382543084, 175.08797571169225, 5.000000000444402, 4.9999999999226645]}


**Configure Instance Dataset**

In [ ]:
n_list = info['n_list']
x_list = info['x_list']
y_list = info['y_list']
c_list = info['C_list']
#print(x_list[0][0])
#print(len(x_list))
#print(len(x_list[0]))
#print(c_list[0])
#print(c_list)

**MainCode : Algorithm Design**

In [ ]:
###############
# Algorithm
###############
def OptCost(x_list, y_list, c):

#############
# compute error
#############
  N = len(x_list)
  err = np.zeros((len(x_list),len(x_list))) #err[i,j]= error of p_i,p_{i+1},...,p_j
  
  x = np.array(x_list)
  y = np.array(y_list)
  xx = x*x
  yy = y*y
  tmp = x*y

  y2 = np.zeros((N,N)) 
  xy = np.zeros((N,N))
  x2 = np.zeros((N,N))
  ysum = np.zeros((N,N))
  xsum = np.zeros((N,N))
  for i in range(N):
    for j in range(i,N):
      y2[i,j] = y2[i,j-1]+yy[j]
      x2[i,j] = x2[i,j-1]+xx[j]
      xy[i,j] = xy[i,j-1]+tmp[j]
      ysum[i,j] = ysum[i,j-1]+y[j]
      xsum[i,j] = xsum[i,j-1]+x[j]
      a = ((j-i+1)*xy[i,j]-xsum[i,j]*ysum[i,j])/((j-i+1)*x2[i,j]-xsum[i,j]*xsum[i,j])
      b = (ysum[i,j]-a*xsum[i,j])/(j-i+1)
      if not i==j:
        err[i,j] = y2[i,j]-2*a*xy[i,j]+a*a*x2[i,j]-2*b*ysum[i,j]+2*a*b*xsum[i,j]+(j-i+1)*b*b

###############
# compete OPT and its index
###############

  OPT = np.ones(len(x_list))*100000000000000 #OPT[i] = min cost of p_0,p_1,...p_i
  OPT[0] = c
  OPT[1] = c
  inds = [[0,[0]] for i in range(len(x_list))]
  inds[0], inds[1] = [0,[]], [0,[]] #inds[i] = [k_i, [id_k]], where k is #lines, id_k is the list of last opint of lines that minimize cost for p_1,p_2,...,p_i 
  for j in range(1, len(x_list)):
    temp = err[0, j] + c
    OPT[j] = temp
    inds[j][0], inds[j][1] = 1, [j]
    for i in range(2,j):
      temp = err[i,j] + c +OPT[i-1] #0-(i-1) + i-j
      if temp<OPT[j]:
        OPT[j] = temp
        inds[j][0], inds[j][1] = inds[i-1][0]+1, inds[i-1][1]+[j]
  return OPT[len(x_list)-1],inds[len(x_list)-1]

**Test Process Based on the Instance_Dataset**

In [ ]:
k_list = []
last_points_list = []
opt_list = []

for i in range(len(n_list)):
  opt, inds = OptCost(x_list[i],y_list[i],c_list[i])
#opt, inds = OptCost(x_list[0],y_list[0],750)
#opt, inds = OptCost(x_list,y_list,750)
  opt_list.append(opt)
  k_list.append(inds[0])
  last_points_list.append(inds[1])
ans = {}
ans.update({'k_list': k_list, 'last_pointes_list': last_points_list, 'opt_list': opt_list})
print(ans)

<ipython-input-7-f5d1d8cd31d7>:30: RuntimeWarning: invalid value encountered in double_scalars
  a = ((j-i+1)*xy[i,j]-xsum[i,j]*ysum[i,j])/((j-i+1)*x2[i,j]-xsum[i,j]*xsum[i,j])


{'k_list': [3, 5, 7, 5, 5, 6, 5, 6, 5, 5], 'last_pointes_list': [[31, 44, 93], [17, 34, 49, 75, 95], [21, 45, 49, 62, 70, 72, 76], [14, 35, 58, 67, 97], [21, 32, 54, 80, 104], [16, 46, 71, 100, 119, 129], [9, 38, 52, 65, 88], [24, 46, 69, 72, 101, 122], [10, 30, 55, 69, 94], [11, 35, 47, 73, 97]], 'opt_list': [9955.201272375149, 12127.705100827889, 9763.325183169176, 10943.8680718462, 124.3173332807646, 165.2299849440905, 124.36266382535104, 175.08797571171417, 5.000000000427761, 4.999999999924673]}


# **Here is the Test Result for 3 dateset**

**Small_solution**

In [ ]:
#################
# upload dataset
#################
import pickle
file = open('/content/drive/MyDrive/629/Project_1/test_set_small_instances', 'rb')
info = pickle.load(file)

In [ ]:
################
# compute each list in dataset
################
n_list = info['n_list']
x_list = info['x_list']
y_list = info['y_list']
c_list = info['C_list']

In [ ]:
################
# based on the alrgiam I desiged, test result 
################
k_list = []
last_points_list = []
opt_list = []

for i in range(len(n_list)):
  opt, inds = OptCost(x_list[i],y_list[i],c_list[i])
  opt_list.append(opt)
  k_list.append(inds[0])
  last_points_list.append(inds[1])
ans = {}
ans.update({'k_list': k_list, 'last_pointes_list': last_points_list, 'opt_list': opt_list})
print(ans)
pickle.dump(ans, open('test_small_solution', "wb"))

<ipython-input-19-6735006a232e>:26: RuntimeWarning: invalid value encountered in double_scalars
  a = ((j-i+1)*xy[i,j]-xsum[i,j]*ysum[i,j])/((j-i+1)*x2[i,j]-xsum[i,j]*xsum[i,j])


{'k_list': [26, 20, 20, 24, 15, 24, 19, 21, 24, 21, 20, 22, 21, 19, 22, 19, 25, 28, 21, 17, 21, 18, 20, 19, 20, 20, 23, 21, 22, 19, 18, 21, 22, 19, 19, 21, 21, 20, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 11, 9, 9, 10, 11, 13, 12, 11, 10, 12, 9, 14, 7, 8, 9, 7, 13, 8, 12, 16, 9, 10, 10, 10, 10, 10, 10, 9, 10, 9, 9, 10, 11, 12, 11, 10, 12, 10, 9, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10], 'last_pointes_list': [[25, 31, 62, 67, 78, 104, 128, 156, 187, 203, 207, 218, 222, 244, 258, 269, 280, 296, 313, 323, 332, 342, 352, 358, 366, 395], [17, 25, 32, 41, 64, 67, 100, 179, 196, 206, 209, 241, 269, 281, 297, 320, 348, 371, 391, 409], [28, 38, 94, 118, 155, 167, 188, 213, 243, 270, 292, 305, 308, 319, 334, 336, 357, 371, 386, 414], [21, 30, 73, 77, 98, 105, 118, 128, 130, 144, 147, 160, 169, 190, 218, 230, 250, 267, 280, 325, 337, 351, 363, 373], [51, 53, 112, 147, 164, 176, 199, 227, 235, 255, 312, 323, 340, 353, 374], [11, 39, 41, 56, 79, 109, 141, 162, 174, 184, 193, 214, 227, 251,

**Medium solution**

In [ ]:
#################
# upload dataset
#################
import pickle
file = open('/content/drive/MyDrive/629/Project_1/test_set_medium_instances', 'rb')
info = pickle.load(file)

In [ ]:
################
# compute each list in dataset
################
n_list = info['n_list']
x_list = info['x_list']
y_list = info['y_list']
c_list = info['C_list']

In [ ]:
################
# based on the alrgiam I desiged, test result 
################
k_list = []
last_points_list = []
opt_list = []

for i in range(len(n_list)):
  opt, inds = OptCost(x_list[i],y_list[i],c_list[i])
  opt_list.append(opt)
  k_list.append(inds[0])
  last_points_list.append(inds[1])
ans = {}
ans.update({'k_list': k_list, 'last_pointes_list': last_points_list, 'opt_list': opt_list})
print(ans)
pickle.dump(ans, open('test_medium_solution', "wb"))

<ipython-input-26-6735006a232e>:26: RuntimeWarning: invalid value encountered in double_scalars
  a = ((j-i+1)*xy[i,j]-xsum[i,j]*ysum[i,j])/((j-i+1)*x2[i,j]-xsum[i,j]*xsum[i,j])


{'k_list': [95, 86, 87, 85, 76, 76, 71, 96, 89, 88, 91, 85, 91, 88, 89, 86, 87, 91, 86, 82, 80, 77, 80, 80, 79, 80, 79, 78, 80, 78, 37, 44, 39, 44, 42, 43, 46, 38, 42, 40, 42, 31, 36, 40, 43, 40, 43, 35, 45, 38, 43, 44, 45, 45, 45, 43, 43, 43, 38, 46, 40, 43, 41, 41, 41, 41, 42, 42, 40, 42, 40, 40, 40, 38, 40, 38, 40, 39, 40, 40], 'last_pointes_list': [[59, 179, 204, 245, 259, 280, 305, 343, 347, 353, 374, 390, 393, 395, 401, 423, 434, 444, 459, 463, 474, 489, 494, 497, 507, 526, 538, 551, 553, 565, 595, 610, 633, 656, 691, 708, 728, 775, 782, 808, 830, 845, 856, 858, 881, 894, 909, 923, 946, 949, 966, 990, 1020, 1035, 1061, 1090, 1117, 1129, 1136, 1141, 1154, 1167, 1193, 1214, 1233, 1244, 1262, 1279, 1289, 1333, 1346, 1358, 1379, 1397, 1411, 1417, 1432, 1448, 1450, 1465, 1492, 1513, 1515, 1527, 1542, 1563, 1566, 1586, 1592, 1602, 1618, 1641, 1647, 1666, 1672], [140, 145, 150, 181, 229, 278, 309, 348, 359, 363, 383, 389, 418, 437, 488, 498, 524, 536, 538, 560, 583, 594, 596, 610, 630, 

**Large solution**

In [ ]:
#################
# upload dataset
#################
import pickle
file = open('/content/drive/MyDrive/629/Project_1/test_set_large_instances', 'rb')
info = pickle.load(file)


In [ ]:
################
# compute each list in dataset
################
n_list = info['n_list']
x_list = info['x_list']
y_list = info['y_list']
c_list = info['C_list']

In [ ]:
################
# based on the alrgiam I desiged, test result 
################
k_list = []
last_points_list = []
opt_list = []

for i in range(len(n_list)):
  opt, inds = OptCost(x_list[i],y_list[i],c_list[i])
  opt_list.append(opt)
  k_list.append(inds[0])
  last_points_list.append(inds[1])
ans = {}
ans.update({'k_list': k_list, 'last_pointes_list': last_points_list, 'opt_list': opt_list})
pickle.dump(ans, open('test_large_solution', "wb"))

<ipython-input-30-8177e65fd4f8>:26: RuntimeWarning: invalid value encountered in double_scalars
  a = ((j-i+1)*xy[i,j]-xsum[i,j]*ysum[i,j])/((j-i+1)*x2[i,j]-xsum[i,j]*xsum[i,j])
